In [2]:
# Import the libraries we need for this lab

# Using the following line code to install the torchvision library
# !mamba install -y torchvision

import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F
import matplotlib.pylab as plt
import numpy as np

In [5]:
# Define function to plot accuracy and loss
def plot_accuracy_loss(training_results):
    plt.subplot(2,1,1)
    plt.plot(training_results['training_loss'],'r')
    plt.ylabel('loss')
    plt.title('training loss iteration')
    plt.subplot(2,1,2)
    plt.ylabel('accuracy')
    plt.xlabel('epochs')
    plt.show()

In [7]:
# Define function to plot model parameters 
def print_model_parameters(model):
    count = 0
    for ele in model.state_dict():
        count +=1
        if count % 2 != 0:
            print("The following are the parameters for layer", count //2+1)
        if ele.find("bias") != -1:
            print("The size of bias:", model.state_dict()[ele].size())
        else:
            print("The size of weight:", model.state_dict()[ele].size())

In [8]:
# Define the neural network module or class :
def show_data(data_sample):
    plt.imshow(data_sample.numpy().reshape(28,28), cmap='gray')
    plt.show()

In [10]:
# Define neural network class 

class Net(nn.Module):

    # Constructor 
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H)
        self.linear2 = nn.Linear(H, D_out)

    # Predictor 
    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

In [38]:
# Define training function to train model 

def train(model, criterion, train_loader, validation_loader, optimizer, epochs=100):
    i = 0
    useful_stuff= {'training_loss':[], 'validation_accuracy':[]}
    for epoch in range(epochs):
        for i, (x,y) in enumerate(train_loader):
            optimizer.zero_grad()
            z = model(x.view(-1,28*28))
            loss = criterion(z,y)
            loss.backward()
            optimizer.step()
            # Loss for every iteration 
            useful_stuff['training_loss'].append(loss.data.item())
        correct = 0
        for x,y in validation_loader:
            # Validation 
            z = model(x.view(-1,28*28))
            _, label = torch.max(z,1)
            correct += (label ==y).sum().item()
        accuracy = 100 * (correct /len(validation_dataset))
        useful_stuff['validation_accuracy'].append(accuracy)
    return useful_sfuff
    

In [39]:
# Make the data 
train_dataset = dsets.MNIST(root='./data', train = True, download=True,transform = transforms.ToTensor())

In [32]:
# Create validation dataset
validation_dataset= dsets.MNIST(root='./data', download=True,transform = transforms.ToTensor())

In [33]:
# Create criterion function 
criterion = nn.CrossEntropyLoss()

In [34]:
# Create data loader for both dataset 
train_loader=  torch.utils.data.DataLoader(dataset=train_dataset, batch_size=2000, shuffle=True)
validation_loader= torch.utils.data.DataLoader(dataset=validation_dataset, batch_size = 2000, shuffle=True)

In [40]:
# Define the neural network, optimizer and train the model 

# Create model with 100 neurons 
input_dim = 28*28 
hidden_dim = 100
output_dim = 10

model = Net(input_dim, hidden_dim, output_dim)

In [41]:
# Print the model parameters 
print_model_parameters(model)

# Set the learning rate and optimizer 
learning_rate= 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

The following are the parameters for layer 1
The size of weight: torch.Size([100, 784])
The size of bias: torch.Size([100])
The following are the parameters for layer 2
The size of weight: torch.Size([10, 100])
The size of bias: torch.Size([10])


In [ ]:
# Train the model 
training_results = train(model, criterion, train_loader, validation_loader, optimizer, epochs =30 )

In [ ]:
# Analyze the results 

# plot accuracy and loss 
plot_accuracy_loss(training_results)

# Plot the first five misclassified samples 

count=0
for x,y in validation_dataset:
    z = model(x.reshape(-1, 28*28))
    _, yhat = torch.max(z,1)
    if yhat != y:
        show_data(x)
        count += 1
    if count >= 5:
        break